Code for advecting a passive tracer

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scienceplots
plt.style.use('science')
from grid_interpolation import *

# Functions
def generate_velocity(streamfunction, y, x):
    N = np.shape(y)[1]  # length in the y direction
    M = np.shape(x)[0]
    dx = dy = y[0, 1] - y[0, 0]  # extract grid space step from y array
    
    # Implement centred difference scheme for u, ignoring the boundary 
    # conditions as u is not defined on the boundary
    u = -1 * ((streamfunction[:, 2:N] - 
               streamfunction[:, 0:(N-2)]) / (2 * dy))

    # Define boundary conditions for v
    v_first = (streamfunction[1, 1:(N-1)] - 
               streamfunction[-1, 1:(N-1)]) / (2 * dx)
    
    v_last = (streamfunction[0, 1:(N-1)] - 
              streamfunction[-2, 1:(N-1)]) / (2 * dx)

    # Implement centred difference scheme for middle values of v
    v_interior = (streamfunction[2:M, 1:(N-1)] - 
                  streamfunction[0:(M-2), 1:(N-1)]) / (2 * dx)
    
    # Combine interior and boundary values for v
    v = np.vstack([v_first, v_interior, v_last])

    return u, v

def compute_dep_pts(u, v, dt, x, y, dx, interp_order, max_iterations=100):

    # First we set a convergence threshold
    convergence_threshold = 0.01 * dx

    # Set an initial guess of wind at the midpoint between departure and 
    # arrival points to be the wind at the arrival point
    u_mid, v_mid = u, v

    # Set the first guess of departure points 
    x_dep = x - (u_mid * dt)
    y_dep = y - (v_mid * dt)

    # Iterate, updating departure points until convergence 
    for _ in range(max_iterations):
        # Interpolate wind to departure points using custom interpolation
        u_dep, v_dep = interpolate([u, v], x, y, x_dep, y_dep, 
                                interp_order=interp_order, wrap=[True, False])

        # Estimate wind at midpoint
        u_mid = 0.5 * (u_dep + u)
        v_mid = 0.5 * (v_dep + v)

        # Compute new estimate departure points
        x_dep_new = x_dep - u_mid * dt
        y_dep_new = y_dep - v_mid * dt

        # Compute change from (x_dep, y_dep) to (x_dep_new, y_dep_new) across 
        # the grid
        max_change = np.max(np.sqrt((x_dep_new - x_dep)**2 + 
                                    (y_dep_new - y_dep)**2))

        # Check for convergence
        if max_change < convergence_threshold * dx:
            break

        # Update departure points
        x_dep, y_dep = x_dep_new, y_dep_new

    return x_dep, y_dep

def advect_tracer(num_steps, q, x, y, x_dep, y_dep, interp_order):

    for time_step in range(int(num_steps)):

        # Interpolate q to departure points
        q_dep = interpolate([q[:,1:-1]], x[:,1:-1], y[:,1:-1], x_dep, y_dep,
                             interp_order=interp_order, wrap=[True, False])

        # Compute new q
        q[:,1:-1] = q_dep[0]

    return q

# constants
phi_0     = 1e5
f_0       = 1e-4
beta      = 1.6e-11
x_0_psi   = 1.25e7
x_0_q     = 1.375e7
y_0_psi   = y_0_q = 2.5e6
A         = 3e-2
sigma_psi = 7.5e5
sigma_q   = 5e5

# set up some interpolation orders for experiments
interpolation_order = 5

# duration of simulation
duration  = 6 * 24 * 60 * 60 # 6 days in seconds
dt        = 60 * 60 # 1 hour in seconds
num_steps = duration/dt # number of timesteps

# Define the grid:

# common grid spacing
grid_space = 1e5

# adding 1e5 to both limits so that np.arrange() includes the upper limit
upper_x_limit = 2.5e7 + grid_space
upper_y_limit = 5e6 + grid_space

x = np.arange(0, upper_x_limit, grid_space)
y = np.arange(0, upper_y_limit, grid_space)

# define an X array where :
Y, X = np.meshgrid(y, x)

# As always, we define the streamfunction and passive tracer using the 
# analytical solutions:
psi  = phi_0/f_0 * (1 - 
                    (A * np.exp(-((X-(x_0_psi))**2 + 
                                  (Y-y_0_psi)**2)/(2 * (sigma_psi**2)))))

q_xy = np.exp(-((X-(x_0_q))**2 + (Y-y_0_q)**2)/(2 * (2 * sigma_q**2))) 

# visualise q_xy
fig, ax = plt.subplots(figsize=(7,4), dpi=300)
ax.pcolormesh(q_xy.T)
ax.set_title('$q_{xy}$')
ax.set_aspect('equal')
ax.set_xlabel('X-Gridpoints')
ax.set_ylabel('Y-Gridpoints')
plt.show()

# generate velocity
U, V = generate_velocity(psi, Y, X)

# compute departure points
x_dep, y_dep = compute_dep_pts(u = U, v=V, dt=dt, x=X[:,1:-1], y=Y[:,1:-1], 
                               dx=grid_space, interp_order=interpolation_order)

# advect the tracer field
first_order_advection = advect_tracer(num_steps, q_xy.copy(), X, Y, x_dep, 
                                      y_dep, 1)

third_order_advection = advect_tracer(num_steps, q_xy.copy(), X, Y, x_dep, 
                                      y_dep, 3)

fifth_order_advection = advect_tracer(num_steps, q_xy.copy(), X, Y, x_dep, 
                                      y_dep, 5)

# plot the different advection schemes
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10, 3), dpi=300, sharex=True)

for axis_row in ax:
    for axis in axis_row:
        axis.set_aspect('equal')
        axis.set_xlabel('X-Gridpoints')
        axis.set_ylabel('Y-Gridpoints')

ax[0, 0].pcolormesh(first_order_advection.T)
ax[0, 0].set_title('1$^{st}$ order tracer advection at 6 days')
ax[0, 1].pcolormesh(np.roll(first_order_advection, 32000, axis=0).T)
ax[0, 1].set_title('1$^{st}$ order tracer advection at 6 days, shifted')
ax[1, 0].pcolormesh(third_order_advection.T)
ax[1, 0].set_title('3$^{rd}$ order tracer advection at 6 days')
ax[1, 1].pcolormesh(fifth_order_advection.T)
ax[1, 1].set_title('5$^{th}$ order tracer advection at 6 days')

# Add a colorbar
# cbar_ax = fig.add_axes([0.9, 0.19, 0.02, 0.65])  # [x, y, width, height]
# cbar = fig.colorbar(c1, cax=cbar_ax, label='s$^{-1}$')

plt.xlabel('X-Gridpoints')
plt.ylabel('Y-Gridpoints')
plt.subplots_adjust(hspace=0.2)
plt.show()

# Roll the tracer field to the edge to confirm periodicity in x-direction
fig, ax = plt.subplots(figsize=(7,4), dpi=300)
ax.pcolormesh(np.roll(first_order_advection, 32000, axis=0).T)
ax.set_title('$q_{xy}$')
ax.set_aspect('equal')
ax.set_xlabel('X-Gridpoints')
ax.set_ylabel('Y-Gridpoints')
plt.show()